# 5_serve_online_predictions_with_Cloud_AI_Platform

Once we've already built our Keras model on Cloud AI Platform, we'll be deploying our Keras model to Cloud AI Platform and creating predictions in this notebook.

## Set up environment variables and load necessary libraries

In [25]:
import os

In [26]:
PROJECT = "predict-babyweight-10142021"
BUCKET = PROJECT
REGION = "us-central1"

os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET 
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.1"

In [27]:
%%bash
gcloud config set compute/region $REGION
gcloud config set ai_platform/region global

Updated property [compute/region].
Updated property [ai_platform/region].


## Check our trained model files in GCP bucket

Let's check the directory structure of our outputs of our trained model.

In [28]:
%%bash
gsutil ls gs://${BUCKET}/model_trained_Dataflow_tuned

gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/
gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/20211023150249/
gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/checkpoints/


In [29]:
%%bash
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/model_trained_Dataflow_tuned/2*)
gsutil ls ${MODEL_LOCATION}

gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/20211023150249/
gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/20211023150249/saved_model.pb
gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/20211023150249/assets/
gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/20211023150249/variables/


## Deploy the trained model to GCP

In [30]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="v1"
MODEL_LOCATION="gs://predict-babyweight-10142021/model_trained_Dataflow_tuned/20211023150249/"

gcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model=${MODEL_NAME} \
    --origin=${MODEL_LOCATION} \
    --runtime-version=2.1 \
    --python-version=3.7

Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/predict-babyweight-10142021/models/babyweight].
Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......
.........................................................................................................................................................................................................................................................................................................................................................done.


## Make online prediction

### Use REST API call

We can use the Python API to send a JSON request to the endpoint of the service to make it predict a baby's weight.

In [31]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "babyweight" 
MODEL_VERSION = "v1"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = f"https://ml.googleapis.com/v1/projects/{PROJECT}/models/{MODEL_NAME}/versions/{MODEL_VERSION}:predict" 
headers = {"Authorization": "Bearer " + token }
request_data = {
    "instances": [
        {
        "is_male": "True",
        "mother_age": 26.0,
        "plurality": "Single(1)",
        "gestation_weeks": 39,
        "cigarette_use":"True",
        "alcohol_use": "True"
        },
        {
        "is_male": "False",
        "mother_age": 29.0,
        "plurality": "Single(1)",
        "gestation_weeks": 38,
        "cigarette_use":"False",
        "alcohol_use": "False"
        },
        {
        "is_male": "True",
        "mother_age": 26.0,
        "plurality": "Triplets(3)",
        "gestation_weeks": 39,
        "cigarette_use":"Unknown",
        "alcohol_use": "True"
        },
        {
        "is_male": "Unknown",
        "mother_age": 29.0,
        "plurality": "Multiple(2+)",
        "gestation_weeks": 38,
        "cigarette_use":"Unknown",
        "alcohol_use": "Unknown"
        },
  ]
}
response = requests.post(api, json=request_data, headers=headers)
print(response.content)

b'{"predictions": [{"weight": [7.472408771514893]}, {"weight": [7.3682379722595215]}, {"weight": [4.574194431304932]}, {"weight": [5.919671535491943]}]}'


### Use gcloud shell API

Also, we can use the gcloud shell API. Create a newline delimited JSON file with one instance per line and submit using gcloud.

In [32]:
%%writefile json_samples/small_test.json
{"is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39, "cigarette_use":"True","alcohol_use": "True"}
{"is_male": "False", "mother_age": 29.0, "plurality": "Single(1)", "gestation_weeks": 38,"cigarette_use":"False","alcohol_use": "False"}

Overwriting json_samples/small_test.json


Now call `gcloud ai-platform predict` using the JSON we just created and point to our deployed `model` and `version`.

In [33]:
%%bash
gcloud ai-platform predict \
    --model=babyweight \
    --version=v1 \
    --json-instances=json_samples/small_test.json

WEIGHT
[7.472409725189209]
[7.368237018585205]


Using endpoint [https://ml.googleapis.com/]


## Summary:
In this notebook, we deployed our trained Keras model to Cloud AI Platform and call this deployed model to make predictions via REST API and gcloud shell API.